In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [ ]:
import duckdb 
import pandas as pd
import duck_db_helper

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/exploitation_zone/exploitation_zone.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()

In [ ]:
df_list_names

In [ ]:
dff = df_list[0] # Matches
dfp = df_list[1] # Players  

In [ ]:
dff

In [ ]:
dfp

# Join the matches and football players tables
Now we merge the matches data into each football player statistics on that match

In [ ]:
dfr_home = pd.merge(dfp, dff, left_on=['team_x', 'match_date'], right_on=['HomeTeam', 'Date'], how='inner')
dfr_away = pd.merge(dfp, dff, left_on=['team_x', 'match_date'], right_on=['AwayTeam', 'Date'], how='inner')

In [ ]:
dfr = pd.concat([dfr_home, dfr_away], ignore_index=True)

# Project only the wanted columns for the analysis

First we filter all unused columns that are or redundant or don't have any real value for analysis. We also remove all the betting odds and just keep an average (that was calculated in previous steps). 

In [ ]:
unused_cols = ['match_date', 'fixture', 'kickoff_time','transfers_balance','transfers_in','transfers_out',
               'Div','Date','Time','HomeTeam', 'AwayTeam', 'HTHG', 'HTAG', 'HTR', 'Referee',
               'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR',
                ]

# Check if they exists before droping
unused_cols = [col for col in unused_cols if col in dfr.columns]
dfr = dfr.drop(unused_cols, axis=1)

unused_betting_odds = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 
                        'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA',
                        'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 
                        'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 
                        'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 
                        'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 
                        'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 
                        'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 
                        'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 
                        'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA'
                        ]
unused_betting_odds = [col for col in unused_betting_odds if col in dfr.columns]
dfr = dfr.drop(unused_betting_odds, axis=1)


In [ ]:
dfr

# Select the valid players

We are going to only select the players that have played at least 10 minutes in 10 matches for this analysis.

In [ ]:
dfr_10 = dfr[dfr['minutes'] >= 10]
dfr_10

In [ ]:
name_counts = dfr_10.groupby('name').size()
at_least_10_games = name_counts[name_counts >= 10]

In [ ]:
wanted_players = at_least_10_games.index.to_list()
dfr_only_wanted_players = dfr[dfr['name'].isin(wanted_players)]

In [ ]:
dfr_only_wanted_players

In [ ]:
dfr_only_wanted_players.to_pickle('../data/analytical_sandboxes/analytical_sandbox.pkl')

In [ ]:
dfr_only_wanted_players.to_csv('../data/analytical_sandboxes/analytical_sandbox.csv', index=False)